In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import sys
import numpy as np
import tensorflow as tf
import glob
import random
import matplotlib.pyplot as plt
import time
import pandas as pd
import csv
from models import *
from utils import *

In [2]:
file_list = os.listdir("/data-pool/data_no_backup/ga63cun/hemorrhage_sparse_neighbouring/original/")
file_list[:10]

['ID_7bf7fc943.png.npy',
 'ID_6e8f157bc.png.npy',
 'ID_261bf7f0f.png.npy',
 'ID_870885721.png.npy',
 'ID_fe26bbfeb.png.npy',
 'ID_07b7cada3.png.npy',
 'ID_5a59802b1.png.npy',
 'ID_2401c2a6b.png.npy',
 'ID_82e7e86dd.png.npy',
 'ID_1288eb0dd.png.npy']

In [6]:
#specify training parameters

nr_folds=5
for k in range(0, nr_folds):
    
    #path where trainingdata is located
    path = os.path.abspath("./Data/dataset_neighbouring/")
    initial_lr=5e-4
    beta_1 = 0.9
    beta_2 = 0.999
    batchsize = 32
    
    #initialize data_generators
    df_train = pd.read_csv(os.path.abspath("./model_weights/EfficientNet/") + f'/split_{k}/train.csv')[:1000]
    df_val = pd.read_csv(os.path.abspath("./model_weights/EfficientNet/") + f'/split_{k}/val.csv')[:1000]

    print('Fold: ', k)
    

    gen_train = DataGen(df=df_train, path=path, batch_size=batchsize, augmentation=True, train=True)
    gen_val = DataGen(df=df_val, path=path, batch_size=batchsize, augmentation=False, train=True)
    
    steps = gen_train.__len__()
    epochs = 15
    save_path = os.path.abspath("./model_weights/EfficientNet/") + f'/split_{k}/'
    callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=save_path + 'model_{epoch:02d}.h5', verbose=1, period=1, save_weights_only=True),
                 tf.keras.callbacks.CSVLogger(save_path + '/log.csv', append=True, separator=',')]

    scheduler = tf.keras.experimental.CosineDecayRestarts(
        initial_learning_rate=initial_lr, first_decay_steps=steps, t_mul=2, alpha=0.02)

    model_optimizer = keras.optimizers.Adam(learning_rate=scheduler, beta_1=beta_1, beta_2=beta_2)

    loss_function = tf.losses.BinaryCrossentropy(from_logits=False)

    model = effnet()

    model.compile(optimizer=model_optimizer, loss=loss_function, metrics=['accuracy', 'binary_crossentropy'])

    model.fit(gen_train, epochs=epochs, validation_data=gen_val, callbacks=callbacks)

Fold:  0
cannot load file - substitute data with random slice
cannot load file again - substitute data with random slice


FileNotFoundError: [Errno 2] No such file or directory: '/users/Thalhammer/Sparse-View-Cranial-CT-Reconstruction/Data/dataset_neighbouringID_fa1b4cc41.png.npy'